In [1]:
import pandas as pd
import numpy as np
import requests
import json
import ast
from bs4 import BeautifulSoup

In [14]:
restaurant_data_df = pd.read_excel("rw_restaurants_2025-07_v5.xlsx")
restaurant_data_df

,borough,neighborhood,primaryCategory,primaryLocation,ctaButtons,tags,shortTitle,slug,image,summary,website,menuFileUrl,collections,mealTypes,restaurantInclusionWeek,ecommerce,rest_week_site_url,image_url,partnerId
0,Manhattan,The Seaport,Restaurants,The Seaport,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'The Seaport']",Frenchman's Dough,frenchmans-dough-at-tin-building-by-jean-georges,{'url': '//images.ctfassets.net/1aemqu6a6t65/5...,The Frenchman’s Dough is a traditional Italian...,https://www.tinbuilding.com/restaurants-and-ba...,NaN,[],"['$30 Lunch Price', '$45 Dinner Price', '$45 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/fre...,https://images.ctfassets.net/1aemqu6a6t65/5cF3...,NaN
1,Brooklyn,Bay Ridge,Restaurants,Bay Ridge,"{'primary': {'label': {'presale': False, 'gene...","['Steakhouse', 'Bay Ridge']",Hunter's Steak & Ale House,hunters-steak-ale-house,{'url': '//images.ctfassets.net/1aemqu6a6t65/6...,Enjoy traditional dishes like homemade crab ca...,https://www.hunterssteakhouse.com,NaN,[],"['$30 Lunch Price', '$45 Dinner Price', '$30 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/hun...,https://images.ctfassets.net/1aemqu6a6t65/6j9y...,NaN
2,Manhattan,Chelsea,Restaurants,Chelsea,"{'primary': {'label': {'presale': False, 'gene...","['Japanese / Sushi', 'Chelsea']",Konban,konban,{'url': '//images.ctfassets.net/1aemqu6a6t65/3...,Konban is a contemporary Japanese restaurant o...,https://www.konbannyc.com,NaN,[],['$60 Dinner Price'],['Week 2 (July 28 - Aug 3)'],NaN,https://www.nyctourism.com/restaurant-week/konban,https://images.ctfassets.net/1aemqu6a6t65/31Gx...,NaN
3,Manhattan,Murray Hill,Restaurants,Murray Hill,"{'primary': {'label': {'presale': False, 'gene...","['American (New)', 'Murray Hill']",The Flatiron Room - Murray Hill,the-flatiron-room-murray-hill,{'url': '//images.ctfassets.net/1aemqu6a6t65/7...,The experience at The Flatiron Room - Murray H...,https://theflatironroom.com/murray-hill-home,NaN,[],"['$60 Dinner Price', '$60 Sunday Dinner Price']","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/the...,https://images.ctfassets.net/1aemqu6a6t65/7Dl0...,NaN
4,Manhattan,Noho,Restaurants,Noho,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'Noho']",Tucci,tucci,{'url': '//images.ctfassets.net/1aemqu6a6t65/4...,"Tucci, New York's newest Italian restaurant, s...",https://www.tuccinyc.com,NaN,['for-the-foodies'],"['$60 Dinner Price', '$60 Sunday Dinner Price']","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/tucci,https://images.ctfassets.net/1aemqu6a6t65/42sE...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,Manhattan,Midtown West,Restaurants,Midtown West,"{'primary': {'label': {'presale': False, 'gene...","['Seafood', 'Midtown West']",Docks Off 5th,docks-off-5th,"{'url': '', 'alt': ''}","This seafood-centric restaurant, steps away fr...",https://docksoff5th.com,NaN,[],"['$45 Lunch Price', '$60 Dinner Price', '$45 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/doc...,NaN,1372909.0
625,Manhattan,Lower East Side,Restaurants,Lower East Side,"{'primary': {'label': {'presale': False, 'gene...","['Korean', 'Lower East Side']",8282,8282,"{'url': '', 'alt': ''}","8282 is a light-hearted, fresh take on Korean ...",https://www.eightwoeightwo.com,NaN,['for-the-foodies'],"['$60 Dinner Price', '$60 Sunday Dinner Price']","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/8282,NaN,NaN
626,Manhattan,Harlem,Restaurants,Harlem,"{'primary': {'label': {'presale': False, 'gene...","['Spanish', 'Harlem']",Enoteca Harlem,enoteca-harlem,"{'url': '', 'alt': ''}",Enoteca Harlem was born from a jo

In [9]:
def get_page_data(rest_week_site_url):
    page_data = {}
    req = requests.get(rest_week_site_url)
    if req.status_code != 200:
        return {"location_coords": {"lat": "", "lon": ""}, "pdf_url": "", "venueAddress": ""}
        
    soup = BeautifulSoup(req.text)
    next_data = soup.select("#__NEXT_DATA__")[0]
    next_data_json = json.loads(next_data.text)
    fields = next_data_json["props"]["pageProps"]["pageFields"]["fields"]
    page_data["location_coords"] = fields["location"]
    page_data["pdf_url"] = next_data_json["props"]["pageProps"]["page"].get("fileUrl", "")
    page_data["venueAddress"] = fields["venueAddress"]

    return page_data

In [10]:
def get_all_rest_data(search_data):
    page_data = get_page_data(search_data["rest_week_site_url"])
    
    rest_data = {}
    rest_data["name"] = search_data["shortTitle"]
    rest_data["pdf_url"] = page_data["pdf_url"]
    rest_data["slug"] = search_data["slug"]
    rest_data["url"] = search_data["rest_week_site_url"]
    rest_data["venueAddress"] = page_data["venueAddress"]
    rest_data["latitude"] = page_data["location_coords"]["lat"]
    rest_data["longitude"] = page_data["location_coords"]["lon"]
    rest_data["summary"] = search_data["summary"]
    rest_data["website"] = search_data["website"]
    rest_data["collections"] = search_data["collections"]
    rest_data["borough"] = search_data["borough"]
    rest_data["neighborhood"] = search_data["neighborhood"]
    rest_data["primaryCategory"] = search_data["primaryCategory"]
    rest_data["primaryLocation"] = search_data["primaryLocation"]
    rest_data["restaurantInclusionWeek"] = search_data["restaurantInclusionWeek"]
    rest_data["image_url"] = search_data["image_url"]
    rest_data["partnerId"] = search_data["partnerId"]
    
    mealTypes = ast.literal_eval(search_data["mealTypes"])
    rest_data["meal_type"] = "; ".join(mealTypes).replace(" Price", "")

    rest_tags = ast.literal_eval(search_data["tags"].replace("None", '""'))
    rest_data["tags"] = "; ".join(rest_tags).strip("; ")

    return rest_data

In [12]:
%%time
count = 1
total = len(restaurant_data_df)
all_rest_data = []
for index, restaurant in restaurant_data_df.iterrows():
    print(f"{count} out of {total}: {restaurant['shortTitle']}")
    rest_data = get_all_rest_data(restaurant)
    all_rest_data.append(rest_data)
    count += 1

1 out of 629: Frenchman's Dough
2 out of 629: Hunter's Steak & Ale House
3 out of 629: Konban
4 out of 629: The Flatiron Room - Murray Hill
5 out of 629: Tucci
6 out of 629: Addictive NYC
7 out of 629: Sake No Hana
8 out of 629: Steak Frites Bistro
9 out of 629: Wagamama - Nomad/Flatiron
10 out of 629: 5 Napkin Burger - Hell's Kitchen
11 out of 629: Del Friscos Double Eagle Steakhouse New York
12 out of 629: Wayan
13 out of 629: Aqua New York
14 out of 629: Mezze on the River
15 out of 629: BLT Prime
16 out of 629: Momoya Soho
17 out of 629: Perry Street
18 out of 629: Souvlaki GR - Midtown East
19 out of 629: Leonetta
20 out of 629: Lillie's Victorian—Union Square
21 out of 629: Kru
22 out of 629: The Bronx Beer Hall
23 out of 629: Cull & Pistol Oyster Bar
24 out of 629: Koi New York
25 out of 629: Central Park Boathouse
26 out of 629: Gyu-Kaku Japanese BBQ - Midtown
27 out of 629: Sarabeth's Central Park South
28 out of 629: Jazba
29 out of 629: Jue Lan Club
30 out of 629: Wagamama -

255 out of 629: Langan's
256 out of 629: Jiang Nan - Long Island City
257 out of 629: Hearth
258 out of 629: Frankie & Johnnie’s Steakhouse
259 out of 629: Stout NYC Financial District
260 out of 629: Bar56
261 out of 629: The Independent
262 out of 629: Bar Sprezzatura
263 out of 629: Altesi Downtown
264 out of 629: Benjamin Steakhouse
265 out of 629: La Voglia
266 out of 629: Souvlaki GR - Lower East Side
267 out of 629: Palermo Argentinian Bistro - Gramercy
268 out of 629: JoJo
269 out of 629: Pasta Lovers
270 out of 629: Sant Ambroeus—Brookfield
271 out of 629: Bixi
272 out of 629: Tinto Tapas Restaurant & Wine Bar
273 out of 629: Sarabeth's Park Avenue South
274 out of 629: Wonderland Bar
275 out of 629: Virgil's Real BBQ - Times Square
276 out of 629: Empire Steak House - Midtown East
277 out of 629: STK Rooftop
278 out of 629: Oceans
279 out of 629: Maiella
280 out of 629: Zuma
281 out of 629: Dinner Party
282 out of 629: The Sea Fire Grill
283 out of 629: 53
284 out of 629: Bob

510 out of 629: Oda House - Manhattan
511 out of 629: Stout NYC Bryant Park
512 out of 629: Benjamin Prime
513 out of 629: Tao Uptown
514 out of 629: Palermo Argentinian Bistro - NYC
515 out of 629: STK Midtown
516 out of 629: The Clocktower
517 out of 629: Kissaki
518 out of 629: Felice 64
519 out of 629: Copinette
520 out of 629: Gaonnuri
521 out of 629: The Fulton
522 out of 629: Gyu-Kaku Japanese BBQ - Times Square
523 out of 629: Villa Berulia
524 out of 629: Jacob's Pickles
525 out of 629: Blue Mezze Bar - UES
526 out of 629: HanGawi
527 out of 629: Bananas
528 out of 629: Dinosaur Bar-B-Que - Harlem
529 out of 629: Ophelia Lounge NYC
530 out of 629: Freemans
531 out of 629: Meadowsweet
532 out of 629: Indian Table
533 out of 629: Tonchin Brooklyn
534 out of 629: Mastro's Steakhouse NYC
535 out of 629: Patrick's on the Hill
536 out of 629: Y Space BBQ
537 out of 629: Aquarelle
538 out of 629: Le Monde
539 out of 629: Vetro Restaurant & Lounge
540 out of 629: Nisí Estiatorio
541 o

In [15]:
all_rest_data_df = pd.DataFrame(all_rest_data)
all_rest_data_df["partnerId"] = all_rest_data_df["partnerId"].astype("Int64")

In [17]:
all_rest_data_df.to_excel("rw_restaurants_2025-07_full_v4.xlsx", index=False)